Large time window 25ns

small time window 3.125ns

In [1]:
import ROOT
import time

Welcome to JupyROOT 6.22/02


### Class definitions

In [2]:
class noise_generator:
    def __init__(self, treshold, channel_count):
        noise = ROOT.TF1("noise_gauss", "TMath::Gaus(x, 0, 5)", 0, 1000)
        self._mean = channel_count * noise.Integral(treshold, 1000)/noise.Integral(0, 1000)/2
        self._prng = ROOT.TRandom3(time.time_ns())

    def generate(self):
        total = self._prng.Poisson(self._mean)
        narrow = 0
        for i in range(total):
            if self._prng.Uniform(25) < 3.125:
                narrow += 1
        estimate = 3.125 * (total - narrow)/(25 - 3.125)
        return narrow, estimate

    
class signal_generator:
    def __init__(self, treshold, expected_voltage):
        self._treshold = treshold
        self._expected_voltage = expected_voltage
        signal_func = ROOT.TF1("sth", "TMath::Landau(x, %f, %f)" % (expected_voltage, expected_voltage/4), -100, 1000)
        self.fraction_above = signal_func.Integral(treshold, 1000)/signal_func.Integral(-100, 1000)
        self._prng = ROOT.TRandom3(time.time_ns())
    
    def generate(self, expected_count):
        signal_count = self._prng.Poisson(expected_count)
        above_treshold_count = 0
        for i in range(signal_count):
            if self._prng.Landau(self._expected_voltage, self._expected_voltage/4) > self._treshold:
                above_treshold_count += 1
        return signal_count, above_treshold_count

### Paramaters and return values explained

In [3]:
# Create a noise generator by passing treshold and channel count as paramaters
my_noise_generator = noise_generator(20, 3.6 * 1e6)

# The first return value is the exact number of noise activated channels in the small window and
# the second return value is the estimated number of noise activated channels in the small window.
noise_exact, noise_est = my_noise_generator.generate()

print(noise_exact, noise_est)

# Create a signal generator, the first paramater is the treshold, the second paramater is the
# expected peak voltage
my_signal_generator = signal_generator(20, 100)

# The paramater is the expected number of particles that reach the detector.
# The first return value is the number of particles that reached the detector and the second return
# value is the number of activated channels.
signal_total, signal_above = my_signal_generator.generate(30)

print(signal_total, signal_above)

10 12.428571428571429
29 29


### Minimum working example
This is how I imagine this will be used for a simulation of bunch crossings

In [4]:
channel_count = 3.6 * 1e6
bunch_crossing_count = 10000
treshold = 20
peak_signal_voltage = 100
avg_particles_reaching_decector = 30

noise = noise_generator(treshold, channel_count)
signal = signal_generator(treshold, peak_signal_voltage)

actual_signal_sum = 0
estimated_signal_sum = 0

for i in range(bunch_crossing_count):
    noise_exact, noise_est = noise.generate()
    signal_total, signal_above = signal.generate(avg_particles_reaching_decector)
    
    total_activations = signal_above + noise_exact
    signal_est = (total_activations - noise_est)/signal.fraction_above
    
    actual_signal_sum += signal_total
    estimated_signal_sum += signal_est
    
print(estimated_signal_sum/actual_signal_sum)

0.9991387214709938
